In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import datetime
import json

In [2]:
metadata = pd.read_csv('movies_dataset/movies_metadata.csv')
credits = pd.read_csv('movies_dataset/credits.csv')

/home/pirate/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


# Processing for Sankey diagram

In [4]:
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
meta_ok = metadata[~metadata["id"].str.contains("-")]
meta_ok["id"] = pd.to_numeric(meta_ok["id"])
meta_ok.head()

/home/pirate/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
yolo = meta_ok.join(credits.set_index("id"), on='id')

In [7]:
yolo

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de..."
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de..."


In [8]:
genres_map = {}
g_i = 0

In [9]:
actor_nb_film = {}


for index, row in yolo[yolo["revenue"] > 1000].iterrows():
    cast = ast.literal_eval(row["cast"])
    
    genres = ast.literal_eval(row["genres"])
    
    for a in cast:
        if a["id"] not in actor_nb_film:
            actor_nb_film[a["id"]] = {"counts" : {}, "global_count": 1, "details" : a}
        else:
            actor_nb_film[a["id"]]["global_count"] += 1
            
        for g in genres:
            if g["name"] not in genres_map:
                genres_map[g["name"]] = g_i
                g_i += 1
            
            if g["name"] in actor_nb_film[a["id"]]["counts"]:
                actor_nb_film[a["id"]]["counts"][g["name"]] += 1
            else:
                actor_nb_film[a["id"]]["counts"][g["name"]] = 1  
                              

actor_nb_film

{31: {'counts': {'Animation': 6,
   'Comedy': 26,
   'Family': 8,
   'Drama': 26,
   'Romance': 8,
   'History': 4,
   'War': 1,
   'Horror': 1,
   'Thriller': 9,
   'Fantasy': 5,
   'Crime': 5,
   'Adventure': 3,
   'Action': 3,
   'Science Fiction': 3,
   'Mystery': 2,
   'Documentary': 1},
  'global_count': 45,
  'details': {'cast_id': 14,
   'character': 'Woody (voice)',
   'credit_id': '52fe4284c3a36847f8024f95',
   'gender': 2,
   'id': 31,
   'name': 'Tom Hanks',
   'order': 0,
   'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}},
 12898: {'counts': {'Animation': 4,
   'Comedy': 14,
   'Family': 10,
   'Fantasy': 3,
   'Drama': 1,
   'Science Fiction': 1,
   'Action': 2,
   'Thriller': 1,
   'Adventure': 3,
   'Romance': 1},
  'global_count': 14,
  'details': {'cast_id': 15,
   'character': 'Buzz Lightyear (voice)',
   'credit_id': '52fe4284c3a36847f8024f99',
   'gender': 2,
   'id': 12898,
   'name': 'Tim Allen',
   'order': 1,
   'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzg

In [10]:
directors_nb_film = {}

for index, row in yolo[yolo["revenue"] > 1000].iterrows():
    crew = ast.literal_eval(row["crew"])
    
    genres = ast.literal_eval(row["genres"])
    
    for d in crew:
        if d["job"] != "Director":
            continue
        
        if d["id"] not in directors_nb_film:
            directors_nb_film[d["id"]] = {"counts" : {}, "global_count": 1, "details" : d}
        else:
            directors_nb_film[d["id"]]["global_count"] += 1
            
        for g in genres:
            if g["name"] not in genres_map:
                genres_map[g["name"]] = g_i
                g_i += 1
            
            if g["name"] in directors_nb_film[d["id"]]["counts"]:
                directors_nb_film[d["id"]]["counts"][g["name"]] += 1
            else:
                directors_nb_film[d["id"]]["counts"][g["name"]] = 1  
                              

directors_nb_film

{7879: {'counts': {'Animation': 5, 'Comedy': 5, 'Family': 5, 'Adventure': 3},
  'global_count': 5,
  'details': {'credit_id': '52fe4284c3a36847f8024f49',
   'department': 'Directing',
   'gender': 2,
   'id': 7879,
   'job': 'Director',
   'name': 'John Lasseter',
   'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}},
 4945: {'counts': {'Adventure': 6,
   'Fantasy': 2,
   'Family': 4,
   'Science Fiction': 5,
   'Animation': 1,
   'Comedy': 1,
   'Action': 3,
   'Thriller': 1,
   'Western': 1},
  'global_count': 7,
  'details': {'credit_id': '52fe44bfc3a36847f80a7c7d',
   'department': 'Directing',
   'gender': 2,
   'id': 4945,
   'job': 'Director',
   'name': 'Joe Johnston',
   'profile_path': '/fok4jaO62v5IP6hkpaaAcXuw2H.jpg'}},
 2178: {'counts': {'Comedy': 1, 'Drama': 1, 'Romance': 1},
  'global_count': 1,
  'details': {'credit_id': '52fe44779251416c91011acb',
   'department': 'Directing',
   'gender': 2,
   'id': 2178,
   'job': 'Director',
   'name': 'Forest Whitaker',
   'prof

In [11]:
final_actor_list = sorted(actor_nb_film.items(), key=lambda x: x[1]["global_count"], reverse=True)[:15]
final_director_list = sorted(directors_nb_film.items(), key=lambda x: x[1]["global_count"], reverse=True)[:15]

In [12]:
json_output = {"nodes" : [], "links" : []}

In [13]:
for g, g_i in genres_map.items():
    json_output["nodes"].append({"name" : g})
    
json_output

{'nodes': [{'name': 'Animation'},
  {'name': 'Comedy'},
  {'name': 'Family'},
  {'name': 'Adventure'},
  {'name': 'Fantasy'},
  {'name': 'Drama'},
  {'name': 'Romance'},
  {'name': 'Action'},
  {'name': 'Crime'},
  {'name': 'Thriller'},
  {'name': 'History'},
  {'name': 'Science Fiction'},
  {'name': 'Mystery'},
  {'name': 'Music'},
  {'name': 'Horror'},
  {'name': 'War'},
  {'name': 'Documentary'},
  {'name': 'Western'},
  {'name': 'Foreign'},
  {'name': 'TV Movie'}],
 'links': []}

In [14]:
new_id = len(genres_map)
for a_i, details in final_actor_list:
    json_output["nodes"].append({"name" : details["details"]["name"]})
    for g, subcount in details["counts"].items():
        json_output["links"].append({"source" : new_id, "target": genres_map[g], "value" : subcount})
        
        
    new_id += 1
    
json_output

{'nodes': [{'name': 'Animation'},
  {'name': 'Comedy'},
  {'name': 'Family'},
  {'name': 'Adventure'},
  {'name': 'Fantasy'},
  {'name': 'Drama'},
  {'name': 'Romance'},
  {'name': 'Action'},
  {'name': 'Crime'},
  {'name': 'Thriller'},
  {'name': 'History'},
  {'name': 'Science Fiction'},
  {'name': 'Mystery'},
  {'name': 'Music'},
  {'name': 'Horror'},
  {'name': 'War'},
  {'name': 'Documentary'},
  {'name': 'Western'},
  {'name': 'Foreign'},
  {'name': 'TV Movie'},
  {'name': 'Samuel L. Jackson'},
  {'name': 'Robert De Niro'},
  {'name': 'Bruce Willis'},
  {'name': 'Morgan Freeman'},
  {'name': 'Liam Neeson'},
  {'name': 'Steve Buscemi'},
  {'name': 'Willem Dafoe'},
  {'name': 'Matt Damon'},
  {'name': 'Sylvester Stallone'},
  {'name': 'Nicolas Cage'},
  {'name': 'John Goodman'},
  {'name': 'Robin Williams'},
  {'name': 'J.K. Simmons'},
  {'name': 'Julianne Moore'},
  {'name': 'Brad Pitt'}],
 'links': [{'source': 20, 'target': 7, 'value': 43},
  {'source': 20, 'target': 9, 'value': 

In [15]:
new_id = len(genres_map) + len(final_actor_list)

for d_i, details in final_director_list:
    json_output["nodes"].append({"name" : details["details"]["name"]})
    for g, subcount in details["counts"].items():
        json_output["links"].append({"source" : genres_map[g], "target": new_id, "value" : subcount})
        
        
    new_id += 1
    
json_output

{'nodes': [{'name': 'Animation'},
  {'name': 'Comedy'},
  {'name': 'Family'},
  {'name': 'Adventure'},
  {'name': 'Fantasy'},
  {'name': 'Drama'},
  {'name': 'Romance'},
  {'name': 'Action'},
  {'name': 'Crime'},
  {'name': 'Thriller'},
  {'name': 'History'},
  {'name': 'Science Fiction'},
  {'name': 'Mystery'},
  {'name': 'Music'},
  {'name': 'Horror'},
  {'name': 'War'},
  {'name': 'Documentary'},
  {'name': 'Western'},
  {'name': 'Foreign'},
  {'name': 'TV Movie'},
  {'name': 'Samuel L. Jackson'},
  {'name': 'Robert De Niro'},
  {'name': 'Bruce Willis'},
  {'name': 'Morgan Freeman'},
  {'name': 'Liam Neeson'},
  {'name': 'Steve Buscemi'},
  {'name': 'Willem Dafoe'},
  {'name': 'Matt Damon'},
  {'name': 'Sylvester Stallone'},
  {'name': 'Nicolas Cage'},
  {'name': 'John Goodman'},
  {'name': 'Robin Williams'},
  {'name': 'J.K. Simmons'},
  {'name': 'Julianne Moore'},
  {'name': 'Brad Pitt'},
  {'name': 'Steven Spielberg'},
  {'name': 'Clint Eastwood'},
  {'name': 'Woody Allen'},
  {'

In [16]:
with open('skankey.json', 'w') as outfile:
    json.dump(json_output, outfile)

# Processing for race

In [17]:
init_data = pd.read_csv("data_cumulative_race.csv")

In [18]:
init_data

,date,name,value
0,1874,Documentary,1
1,1878,Documentary,2
2,1883,Documentary,3
3,1887,Documentary,4
4,1888,Documentary,6
...,...,...,...
2695,1959,TV Movie,0
2696,1960,TV Movie,0
2697,1962,TV Movie,1
2698,2018,TV Movie,766


In [19]:
for i, test in init_data[init_data["name"] == "Documentary"].iterrows():
    print(test["date"], test["value"])

1874 1
1878 2
1883 3
1887 4
1888 6
1890 7
1891 12
1894 22
1895 26
1896 32
1897 33
1898 34
1900 37
1901 41
1903 43
1904 44
1905 45
1909 46
1910 47
1919 48
1922 50
1924 53
1925 55
1926 60
1927 62
1928 66
1929 69
1930 72
1931 74
1933 76
1934 78
1935 79
1936 80
1937 82
1938 84
1939 85
1940 86
1942 90
1943 97
1944 102
1945 108
1946 109
1948 112
1949 113
1950 114
1951 116
1952 118
1953 123
1954 124
1955 127
1956 132
1957 137
1958 142
1959 148
1960 151
1961 155
1962 159
1963 170
1964 179
1965 187
1966 195
1967 207
1968 223
1969 233
1970 252
1971 272
1972 290
1973 302
1974 321
1975 333
1976 351
1977 368
1978 382
1979 395
1980 411
1981 426
1982 440
1983 455
1984 473
1985 492
1986 506
1987 517
1988 537
1989 549
1990 564
1991 580
1992 601
1993 616
1994 643
1995 677
1996 702
1997 740
1998 791
1999 841
2000 893
2001 958
2002 1034
2003 1121
2004 1224
2005 1336
2006 1476
2007 1633
2008 1819
2009 2038
2010 2252
2011 2491
2012 2764
2013 3053
2014 3366
2015 3658
2016 3853
2017 3921
1892 12
1893 12
1899 

In [20]:
default_labels = list(init_data["name"].drop_duplicates())
default_labels.sort()
default_dict = {}

for l in default_labels:
    default_dict[l] = 0

In [21]:
structured_data = {}

for year in range(1874,2020):
    structured_data[year] = {}
    
    current_year_data = init_data[init_data["date"] == year]
    
    if(current_year_data.shape[0] == 0):
        structured_data[year] = structured_data[year - 1]
    else:
        sorted_styles = {}

        for _, row in current_year_data.sort_values(by=['value'], ascending=False).iterrows():
            sorted_styles[row["name"]] = row["value"]
            
        structured_data[year] = sorted_styles

In [22]:
structured_data

{1874: {'Documentary': 1,
  'Comedy': 0,
  'Foreign': 0,
  'Mystery': 0,
  'Western': 0,
  'Music': 0,
  'Adventure': 0,
  'Crime': 0,
  'Thriller': 0,
  'Science Fiction': 0,
  'Family': 0,
  'War': 0,
  'Romance': 0,
  'Horror': 0,
  'Fantasy': 0,
  'History': 0,
  'Drama': 0,
  'Animation': 0,
  'Action': 0,
  'TV Movie': 0},
 1875: {'Documentary': 1,
  'Comedy': 0,
  'Foreign': 0,
  'Mystery': 0,
  'Western': 0,
  'Music': 0,
  'Adventure': 0,
  'Crime': 0,
  'Thriller': 0,
  'Science Fiction': 0,
  'Family': 0,
  'War': 0,
  'Romance': 0,
  'Horror': 0,
  'Fantasy': 0,
  'History': 0,
  'Drama': 0,
  'Animation': 0,
  'Action': 0,
  'TV Movie': 0},
 1876: {'Documentary': 1,
  'Comedy': 0,
  'Foreign': 0,
  'Mystery': 0,
  'Western': 0,
  'Music': 0,
  'Adventure': 0,
  'Crime': 0,
  'Thriller': 0,
  'Science Fiction': 0,
  'Family': 0,
  'War': 0,
  'Romance': 0,
  'Horror': 0,
  'Fantasy': 0,
  'History': 0,
  'Drama': 0,
  'Animation': 0,
  'Action': 0,
  'TV Movie': 0},
 1877: 

In [23]:
new_data = pd.DataFrame(columns=['name', 'value', 'year', 'lastValue', 'rank'])

In [24]:
new_data

,name,value,year,lastValue,rank


In [25]:
def get_new_row_struct():
    return {"name" : "", "value" : 0, "year" : 1900, "lastValue" : 0, "rank" : 0}

In [26]:
for year, ranking in structured_data.items():
    i = 1
    for style, val in ranking.items():
        new_row = get_new_row_struct()
        new_row["name"] = style
        new_row["value"] = val
        new_row["year"] = year
        if (year - 1) in structured_data:
            new_row["lastValue"] = structured_data[year - 1][style]
        else:
            new_row["lastValue"] = val
        new_row["rank"] = i
        i += 1
        
        new_data = new_data.append(new_row, ignore_index=True)

In [27]:
new_data

,name,value,year,lastValue,rank
0,Documentary,1,1874,1,1
1,Comedy,0,1874,0,2
2,Foreign,0,1874,0,3
3,Mystery,0,1874,0,4
4,Western,0,1874,0,5
...,...,...,...,...,...
2915,Music,1598,2019,1598,16
2916,History,1397,2019,1397,17
2917,War,1323,2019,1323,18
2918,Western,1042,2019,1042,19


In [28]:
new_data.to_csv('race_data.csv', index=False)

# Processing for bundle

In [29]:
class Person:

    def __init__(self, uid, name):
        self.uid = uid
        self.name = name
        
    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(other, Person):
            return self.uid == other.uid and self.name == other.name
        return False
    
    def __ne__(self, other):
        """Overrides the default implementation"""
        return not self.__eq__(other)
    
    def __hash__(self):
        """Overrides the default implementation"""
        return self.uid
    
    def __str__(self):
        return "{id : " + str(self.uid) + ", name : " + self.name + "}" 


In [30]:
actors_to_keep = []

for id_a in sorted(actor_nb_film.items(), key=lambda x: x[1]["global_count"], reverse=True)[:50]:
    actors_to_keep.append(id_a[0])

actors_to_keep

[2231,
 380,
 62,
 192,
 3896,
 884,
 5293,
 1892,
 16483,
 2963,
 1230,
 2157,
 18999,
 1231,
 287,
 85,
 4690,
 2283,
 28633,
 13242,
 7399,
 31,
 4038,
 65827,
 6065,
 57755,
 5723,
 1532,
 3,
 17051,
 887,
 1241,
 880,
 4483,
 193,
 2282,
 500,
 7447,
 3036,
 3895,
 2178,
 15831,
 3223,
 1204,
 5049,
 5292,
 3087,
 2975,
 3131,
 2227]

In [31]:
directors_to_keep = []

for id_d in sorted(directors_nb_film.items(), key=lambda x: x[1]["global_count"], reverse=True)[:50]:
    directors_to_keep.append(id_d[0])
    
directors_to_keep

[488,
 190,
 1243,
 578,
 2636,
 1032,
 6159,
 510,
 1150,
 1152,
 5572,
 24,
 1927,
 1884,
 16938,
 2294,
 5140,
 893,
 3026,
 564,
 1723,
 65429,
 7187,
 8246,
 11770,
 1223,
 1776,
 865,
 5281,
 10965,
 5306,
 7396,
 1201,
 39996,
 5216,
 7395,
 8858,
 224,
 638,
 37710,
 1614,
 5174,
 4014,
 3224,
 17494,
 15175,
 19303,
 21370,
 108,
 9181]

In [36]:
direction_map = {}
links_between_artists = []

for index, row in credits.iterrows():
    
    crew = ast.literal_eval(row["crew"])
    cast = ast.literal_eval(row["cast"])
    
    for crew_member in crew:
        if crew_member["job"] == "Director" and crew_member["id"] in directors_to_keep:
            director = Person(crew_member["id"], crew_member["name"])
            if director not in direction_map:
                direction_map[director] = set()
                
    
            for actor in cast:
                if actor["id"] in actors_to_keep:
                    actor_to_add = Person(actor["id"], actor["name"])
                    if actor_to_add != director:
                        direction_map[director].add(actor_to_add)
                        
    for i, actor in enumerate(cast):
        if actor["id"] in actors_to_keep or actor["id"] in directors_to_keep:
            if i < len(cast):
                for j in range(i+1,len(cast)):
                    if cast[j]["id"] in actors_to_keep or cast[j]["id"] in directors_to_keep:
                        links_between_artists.append({"type" : "a2a","source" : actor["id"], "target" : cast[j]["id"]})
                        if actor["id"] == 488 or cast[j]["id"] == 488:
                            print(actor)
                            print()
                            print(cast[j])
                            print()
                            print("------------------------------------")
                            print()

{'cast_id': 2, 'character': 'Dr. Sarah Harding', 'credit_id': '52fe4238c3a36847f800d387', 'gender': 1, 'id': 1231, 'name': 'Julianne Moore', 'order': 1, 'profile_path': '/v2FcWGiiuvl6P7NV0966jNL09uh.jpg'}

{'cast_id': 10, 'character': 'Popcorn-Eating Man', 'credit_id': '52fe4238c3a36847f800d3a7', 'gender': 2, 'id': 488, 'name': 'Steven Spielberg', 'order': 9, 'profile_path': '/pOK15UNaw75Bzj7BQO1ulehbPPm.jpg'}

------------------------------------

{'cast_id': 4, 'character': 'Indiana Jones', 'credit_id': '52fe4215c3a36847f8002c09', 'gender': 2, 'id': 3, 'name': 'Harrison Ford', 'order': 0, 'profile_path': '/7CcoVFTogQgex2kJkXKMe8qHZrC.jpg'}

{'cast_id': 90, 'character': 'Tourist at Airport (uncredited)', 'credit_id': '5895b37e9251415a5b003f39', 'gender': 2, 'id': 488, 'name': 'Steven Spielberg', 'order': 68, 'profile_path': '/pOK15UNaw75Bzj7BQO1ulehbPPm.jpg'}

------------------------------------

{'cast_id': 38, 'character': 'Nigel Powers', 'credit_id': '52fe427bc3a36847f802219b', 'g

{'cast_id': 3, 'character': 'himself', 'credit_id': '52fe4b839251416c751055bd', 'gender': 2, 'id': 108, 'name': 'Peter Jackson', 'order': 1, 'profile_path': '/kSrfI3TbMdIMmPVOflLQ56OfscB.jpg'}

{'cast_id': 4, 'character': 'himself', 'credit_id': '52fe4b839251416c751055c1', 'gender': 2, 'id': 488, 'name': 'Steven Spielberg', 'order': 2, 'profile_path': '/pOK15UNaw75Bzj7BQO1ulehbPPm.jpg'}

------------------------------------

{'cast_id': 4, 'character': 'himself', 'credit_id': '52fe4b839251416c751055c1', 'gender': 2, 'id': 488, 'name': 'Steven Spielberg', 'order': 2, 'profile_path': '/pOK15UNaw75Bzj7BQO1ulehbPPm.jpg'}

{'cast_id': 5, 'character': 'himself', 'credit_id': '52fe4b839251416c751055c5', 'gender': 2, 'id': 510, 'name': 'Tim Burton', 'order': 3, 'profile_path': '/yHEHAHQpN9PfSEQx1UxZPczhcAi.jpg'}

------------------------------------

{'cast_id': 12, 'character': 'Himself', 'credit_id': '53d90243c3a3682c2e002195', 'gender': 2, 'id': 488, 'name': 'Steven Spielberg', 'order': 8, 

In [33]:
pre_json_data = []

for d, cast in direction_map.items():
    tmp = {"director_name" : d.name,
          "key" : d.uid,
           "type" : "cast",
          "cast" : []}
    
    for a in cast:
        tmp["cast"].append({"actor_name": a.name, "key" : a.uid})
        
    #if len(cast) == 0:
        #continue
    
    pre_json_data.append(tmp)

pre_json_data = pre_json_data + links_between_artists

In [34]:
with open('casting.json', 'w') as outfile:
    json.dump(pre_json_data, outfile)

In [35]:
#with open('links_between_artists.json', 'w') as outfile:
    #json.dump(links_between_artists, outfile)